In [0]:
dfCountry = spark.read.json('/covid_data_lake/raw/country/Country.json', mode='overwrite')
dfCountryData = spark.read.json('/covid_data_lake/raw/dados/CountryData.json', mode='overwrite')
dfProvinceData = spark.read.json('/covid_data_lake/raw/dados/ProvinceData.json', mode='overwrite')
dfProvinceCodes = spark.read.json('/covid_data_lake/raw/dados/ProvinceCodes.json', mode='overwrite')

In [0]:
#Transforma os dados
dados1 = dfCountryData.join(dfCountry, dfCountryData.CountryCode == dfCountry.ISO2, how='left')
dfProvinceData = dfProvinceData.join(dfCountry, dfProvinceData.CountryCode == dfCountry.ISO2, how='left')
dados2 = dfProvinceData.join(dfProvinceCodes, dfProvinceData.Province == dfProvinceCodes.Name, how='left')



In [0]:
#Cria View temporária
dados1.createOrReplaceTempView('Dados')  

In [0]:
dados1 = spark.sql('''
select ID, Country, CountryCode, Province, City, CityCode, Lat, Lon, ISO2, Slug, Confirmed, Deaths, Recovered, Active, Date,
(Confirmed - dConfirmed) as dConfirmed, 
(Deaths - dDeaths) as dDeaths, 
(Recovered - dRecovered) as dRecovered 
from (select *,
lag (Confirmed, 1, 0)  over (PARTITION BY CountryCode ORDER BY Date) as dConfirmed, 
lag (Deaths, 1, 0)  over (PARTITION BY CountryCode ORDER BY Date) as dDeaths,
lag (Recovered, 1, 0)  over (PARTITION BY CountryCode ORDER BY Date) as dRecovered
from Dados) tab
''')

In [0]:
from pyspark.sql.types import DateType

dados1 = dados1.withColumn('Date', dados1.Date.cast(DateType()))
dados2 = dados2.withColumn('Date', dados2.Date.cast(DateType()))


In [0]:
dados1 = dados1.drop(*['City', 'CityCode', 'Province', 'Slug', 'ISO2'])
dados2 = dados2.drop(*['Slug', 'ISO2', 'Name'])

In [0]:
#Melhorar performace das querys com base no ANO?

from pyspark.sql.functions import countDistinct
for column in dados1:
  dados1.agg(countDistinct(column)).show()
  
  

+-------------+
count(Active)|
+-------------+
 30397|
+-------------+

+----------------+
count(Confirmed)|
+----------------+
 46639|
+----------------+

+------------------+
count(CountryCode)|
+------------------+
 191|
+------------------+

+-----------+
count(Date)|
+-----------+
 506|
+-----------+

+-------------+
count(Deaths)|
+-------------+
 16350|
+-------------+

+---------+
count(ID)|
+---------+
 96607|
+---------+

+----------+
count(Lat)|
+----------+
 189|
+----------+

+----------+
count(Lon)|
+----------+
 191|
+----------+

+----------------+
count(Recovered)|
+----------------+
 38760|
+----------------+

+--------------+
count(Country)|
+--------------+
 191|
+--------------+

In [0]:
dados1.write.parquet('/covid_data_lake/cleaned/dados/CountryData.parquet', mode='overwrite')
dados2.write.parquet('/covid_data_lake/cleaned/dados/ProvinceData.parquet', mode='overwrite')

In [0]:
#Cria tabelas definitivas
dados1.createOrReplaceTempView('TableDados')
dados2.createOrReplaceTempView('TableProvinces')
